In [1]:
import numpy as np
import pandas as pd
import os
import tomlkit
from glob import glob

import torch

from src.utils import load_data

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

data, genes, feature_names = load_data("MF+METH+GE+SYS+TOPO+scRaw_all+nonzero_mean", "xgb")
data.unlabeled_mask = ~(data.train_mask | data.val_mask | data.test_mask)
data.y_independent = ((data.y_oncokb_latest + data.y_ongene + data.y_tsgene + data.y_ncg7 + data.y_cancermine + data.y_ccgda) > 0).float()
independent_mask = data.unlabeled_mask & data.independent_mask

/db2/users/kyungwooklee/scDiRectX/src/utils.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  array["stat"] = [stat] * len(array)
/db2/users/kyungwooklee/scDiRectX/src/utils.py:176: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 3 on the right)
  x = x.join(pd.DataFrame(sc_x, index=sc_matched_ppi_genes, columns=sc_celltypes))


2983 796 2187


In [4]:
def prk(y, pred, k=100):
    y = y.numpy()
    idx = np.argsort(-pred)

    return y[idx[:k]].sum() / k

def avepk(y, pred, k=100):
    y = y.numpy()
    total = y.sum()
    idx = np.argsort(-pred)
    y = y[idx]
    if len(pred) > k: y = y[:k]

    score = 0
    for i, yval in enumerate(y):
        if yval == 1:
            score += y[:i+1].sum() / (i+1)

    return score / min(total, k)

In [5]:
from sklearn.metrics import average_precision_score

def test(metric, data, k, pred_files_list, index_list):
    result = np.zeros(len(pred_files_list))
    for i, files in enumerate(pred_files_list):
        for pf in files:
            pred = np.loadtxt(pf, delimiter="\t")
            if len(pred) == len(independent_mask):
                pred = pred[independent_mask]

            if k is None:
                result[i] += round(metric((data.y_oncokb_latest.bool() | data.y_ncg7.bool() | data.y_ongene.bool() | data.y_tsgene.bool())[independent_mask], pred) / len(files), 6)

            else:
                result[i] += round(metric((data.y_oncokb_latest.bool() | data.y_ncg7.bool() | data.y_ongene.bool() | data.y_tsgene.bool())[independent_mask], pred, k) / len(files), 6)

    df = pd.DataFrame(result, index=index_list, columns=["OKB+NCG+ONG+TSG"])

    display(df)

In [11]:
xgb_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/xgb/MF+METH+GE+SYS+TOPO+scRaw_all+nonzero_mean/preds/pred_*.txt"))[:50]
mlp_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/mlp/MF+METH+GE+SYS+TOPO+scRaw_all+nonzero_mean/preds/pred_*.txt"))[:50]
tabnet_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/tabnet/MF+METH+GE+SYS+TOPO+scRaw_all+nonzero_mean/preds/pred_*.txt"))[:50]
rf_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/rf/MF+METH+GE+SYS+TOPO+scRaw_all+nonzero_mean_nnmaybe/preds/pred_*.txt"))[:50]
svm_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/svm/MF+METH+GE+SYS+TOPO+scRaw_all+nonzero_mean/preds/pred_*.txt"))[:50]

modig_pred_files = sorted(glob(f"{BASE_DIR}/baselines/MODIG/log/MF+METH+GE/preds/pred_*.txt"))
mrngcn_pred_files = sorted(glob(f"{BASE_DIR}/baselines/MRNGCN/log/MF+METH+GE+TOPO/preds/pred_*.txt"))
mtgcn_pred_files = sorted(glob(f"{BASE_DIR}/baselines/GCN/log/MTGCN/MF+METH+GE+TOPO/preds/pred_*.txt"))
emogi_pred_files = sorted(glob(f"{BASE_DIR}/baselines/GCN/log/EMOGI/MF+METH+GE/preds/pred_*.txt"))
hgdc_pred_files = sorted(glob(f"{BASE_DIR}/baselines/GCN/log/HGDC/MF+METH+GE+SYS/preds/pred_*.txt"))
print(len(xgb_pred_files), len(mlp_pred_files), len(tabnet_pred_files), len(rf_pred_files), len(svm_pred_files), 
      len(modig_pred_files), len(mrngcn_pred_files), len(mtgcn_pred_files), len(emogi_pred_files), len(hgdc_pred_files))

pred_files_list = [xgb_pred_files, mlp_pred_files, tabnet_pred_files, rf_pred_files, svm_pred_files, 
                   modig_pred_files, mrngcn_pred_files, mtgcn_pred_files, emogi_pred_files, hgdc_pred_files]
index_list = ["xgb", "mlp", "tabnet", "rf", "svm", "modig", "mrngcn", "mtgcn", "emogi", "hgdc"]
print("prk")
test(prk, data, 100, pred_files_list, index_list)
print("avepk")
test(avepk, data, 100, pred_files_list, index_list)
print("aupr")
test(average_precision_score, data, None, pred_files_list, index_list)

50 50 50 50 50 50 50 50 50 50
prk


,OncoKB,ONGene,NCG7,Bailey,CancerMine,CCGD,TSGene,IntOGen,MTGCN_OKB,MTGCN_NCG
xgb,0.2018,0.0898,0.6010,0.1264,0.7986,0.3552,0.1686,0.0940,0.1932,0.0
mlp,0.1342,0.1348,0.5628,0.1148,0.7990,0.2922,0.1854,0.0716,0.1284,0.0
tabnet,0.0840,0.0934,0.3644,0.0436,0.6996,0.2020,0.1390,0.0538,0.0806,0.0
rf,0.1280,0.1208,0.5050,0.0988,0.8072,0.5294,0.1912,0.0982,0.1096,0.0
svm,0.1600,0.1300,0.3900,0.0300,0.7600,0.2600,0.1900,0.0900,0.1600,0.0
modig,0.0770,0.0768,0.6450,0.0946,0.6582,0.2208,0.1022,0.0678,0.0762,0.0
mrngcn,0.0198,0.0254,0.1666,0.0000,0.3972,0.0938,0.0522,0.0000,0.0108,0.0
mtgcn,0.1506,0.1372,0.5828,0.1206,0.8208,0.3518,0.1980,0.1346,0.1506,0.0
emogi,0.2108,0.1722,0.3736,0.0724,0.7476,0.2954,0.1994,0.0760,0.2008,0.0
hgdc,0.1374,0.0818,0.6410,0.1576,0.7156,0.2880,0.1466,0.1218,0.1368,0.0


,OKB+NCG,OKB+IntOGen,ONG+TSG,OKB+NCG+ONG,OKB+NCG+ONG+TSG,OKB+Int+NCG,OKB+NCG+ONG+TSG+Int,OKB+NCG+ONG+TSG+CM+CCGD,OKB+NCG+ONG+TSG+CM+CCGD+Int
xgb,0.6560,0.2420,0.2530,0.6948,0.7690,0.6692,0.7822,0.93,0.93
mlp,0.5958,0.1684,0.2788,0.6470,0.7006,0.6070,0.7106,0.95,0.95
tabnet,0.4000,0.1190,0.2162,0.4528,0.5244,0.4066,0.5298,0.96,0.96
rf,0.5342,0.1658,0.3042,0.6178,0.7206,0.5342,0.7206,0.96,0.96
svm,0.4500,0.2100,0.2900,0.5100,0.5900,0.4500,0.5900,0.84,0.84
modig,0.6578,0.1158,0.1688,0.7048,0.7504,0.6680,0.7606,0.89,0.89
mrngcn,0.1786,0.0198,0.0764,0.2032,0.2518,0.1786,0.2518,0.61,0.61
mtgcn,0.6196,0.2270,0.2804,0.6682,0.7384,0.6474,0.7578,0.95,0.95
emogi,0.4742,0.2544,0.3346,0.5560,0.6624,0.4930,0.6724,0.89,0.89
hgdc,0.6756,0.2084,0.1912,0.7000,0.7410,0.6856,0.7510,0.94,0.94


avepk


,OncoKB,ONGene,NCG7,Bailey,CancerMine,CCGD,TSGene,IntOGen,MTGCN_OKB,MTGCN_NCG
xgb,0.046718,0.014627,0.399154,0.019513,0.656635,0.148698,0.032540,0.012942,0.044070,0.0
mlp,0.020835,0.025402,0.346586,0.026010,0.677803,0.095858,0.033917,0.008184,0.019724,0.0
tabnet,0.010336,0.012022,0.143818,0.004792,0.499128,0.050182,0.024725,0.005861,0.009763,0.0
rf,0.022177,0.011934,0.295330,0.013398,0.647303,0.337589,0.045404,0.019164,0.019360,0.0
svm,0.030450,0.041750,0.179400,0.001000,0.582300,0.066450,0.040200,0.008500,0.030450,0.0
modig,0.005196,0.004839,0.435255,0.012207,0.432907,0.043497,0.010481,0.004515,0.005118,0.0
mrngcn,0.000683,0.000690,0.031042,0.000000,0.189864,0.009049,0.006979,0.000000,0.000260,0.0
mtgcn,0.020011,0.017111,0.359888,0.020486,0.661721,0.105586,0.031032,0.017236,0.020011,0.0
emogi,0.083375,0.040713,0.184947,0.004931,0.544491,0.091612,0.032198,0.010082,0.079726,0.0
hgdc,0.012581,0.006195,0.438827,0.034629,0.550472,0.072996,0.019338,0.013537,0.012484,0.0


,OKB+NCG,OKB+IntOGen,ONG+TSG,OKB+NCG+ONG,OKB+NCG+ONG+TSG,OKB+Int+NCG,OKB+NCG+ONG+TSG+Int,OKB+NCG+ONG+TSG+CM+CCGD,OKB+NCG+ONG+TSG+CM+CCGD+Int
xgb,0.460086,0.069570,0.075559,0.519598,0.625095,0.482353,0.650920,0.874529,0.874529
mlp,0.377015,0.032783,0.083485,0.434740,0.487265,0.398251,0.509852,0.920451,0.920451
tabnet,0.170586,0.019728,0.054205,0.215041,0.285242,0.177148,0.291971,0.907986,0.907986
rf,0.342215,0.035950,0.091262,0.417351,0.563482,0.342215,0.563482,0.917218,0.917218
svm,0.228700,0.050550,0.115550,0.300300,0.383450,0.228700,0.383450,0.699390,0.699390
modig,0.449929,0.011353,0.023711,0.494617,0.557714,0.460380,0.569495,0.799050,0.799050
mrngcn,0.034998,0.000683,0.009488,0.041777,0.069804,0.034998,0.069804,0.402067,0.402067
mtgcn,0.397146,0.045425,0.062423,0.452638,0.526317,0.429671,0.554945,0.895397,0.895397
emogi,0.271059,0.120083,0.110808,0.368041,0.487536,0.307801,0.526098,0.822964,0.822964
hgdc,0.465215,0.032122,0.031919,0.490522,0.542506,0.491617,0.570276,0.899560,0.899560


aupr


,OncoKB,ONGene,NCG7,Bailey,CancerMine,CCGD,TSGene,IntOGen,MTGCN_OKB,MTGCN_NCG
xgb,0.091147,0.074940,0.296548,0.057071,0.588434,0.168438,0.113473,0.043243,0.088944,0.000812
mlp,0.077345,0.074177,0.288836,0.059898,0.596022,0.191846,0.109988,0.040154,0.072834,0.002320
tabnet,0.061824,0.063397,0.250317,0.029567,0.549797,0.146068,0.096090,0.032089,0.058419,0.001459
rf,0.083166,0.077811,0.285933,0.055034,0.603281,0.258527,0.119677,0.054244,0.077739,0.001689
svm,0.088529,0.075891,0.241610,0.027595,0.573044,0.149939,0.110926,0.037407,0.086399,0.001550
modig,0.059168,0.064111,0.298940,0.043068,0.581219,0.138872,0.106711,0.041180,0.055992,0.001084
mrngcn,0.026206,0.038410,0.183600,0.009015,0.425556,0.084947,0.061285,0.010315,0.023669,0.000150
mtgcn,0.089971,0.084934,0.300204,0.062568,0.623454,0.160049,0.129194,0.058207,0.087697,0.003218
emogi,0.099837,0.084837,0.244651,0.031445,0.567563,0.143280,0.106424,0.040760,0.100257,0.000508
hgdc,0.088292,0.070634,0.335133,0.077438,0.581974,0.171734,0.105909,0.052200,0.086587,0.006317


,OKB+NCG,OKB+IntOGen,ONG+TSG,OKB+NCG+ONG,OKB+NCG+ONG+TSG,OKB+Int+NCG,OKB+NCG+ONG+TSG+Int,OKB+NCG+ONG+TSG+CM+CCGD,OKB+NCG+ONG+TSG+CM+CCGD+Int
xgb,0.325254,0.110376,0.174284,0.365401,0.424338,0.329001,0.427496,0.0,0.0
mlp,0.315493,0.096574,0.168203,0.354423,0.410375,0.319454,0.413299,0.0,0.0
tabnet,0.274410,0.079417,0.148639,0.310980,0.366107,0.277929,0.368917,0.0,0.0
rf,0.314838,0.106113,0.182661,0.357092,0.420141,0.317996,0.422527,0.0,0.0
svm,0.270791,0.107701,0.169893,0.311447,0.373013,0.274501,0.375731,0.0,0.0
modig,0.321487,0.083813,0.159691,0.357066,0.417156,0.326768,0.421707,0.0,0.0
mrngcn,0.197415,0.032906,0.095317,0.223481,0.266002,0.199334,0.267573,0.0,0.0
mtgcn,0.328468,0.118991,0.194571,0.373180,0.440242,0.333875,0.443835,0.0,0.0
emogi,0.274272,0.118947,0.170603,0.317542,0.378803,0.279486,0.382937,0.0,0.0
hgdc,0.364394,0.115267,0.161909,0.398663,0.450251,0.369125,0.453954,0.0,0.0


In [12]:
xgb_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/xgb/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))[:50]
mlp_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/mlp/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))[:50]
tabnet_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/tabnet/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))[:50]
rf_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/rf/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))[:50]
svm_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/svm/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))[:50]

modig_pred_files = sorted(glob(f"{BASE_DIR}/baselines/MODIG/log/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))
mrngcn_pred_files = sorted(glob(f"{BASE_DIR}/baselines/MRNGCN/log/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))
mtgcn_pred_files = sorted(glob(f"{BASE_DIR}/baselines/GCN/log/MTGCN/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))
emogi_pred_files = sorted(glob(f"{BASE_DIR}/baselines/GCN/log/EMOGI/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))
hgdc_pred_files = sorted(glob(f"{BASE_DIR}/baselines/GCN/log/HGDC/MF+METH+GE+SYS+TOPO/preds/pred_*.txt"))

print(len(xgb_pred_files), len(mlp_pred_files), len(tabnet_pred_files), len(rf_pred_files), len(svm_pred_files),
      len(modig_pred_files), len(mrngcn_pred_files), len(mtgcn_pred_files), len(emogi_pred_files), len(hgdc_pred_files))

pred_files_list = [xgb_pred_files, mlp_pred_files, tabnet_pred_files, rf_pred_files, svm_pred_files,
                   modig_pred_files, mrngcn_pred_files, mtgcn_pred_files, emogi_pred_files, hgdc_pred_files]
index_list = ["xgb_nosc", "mlp_nosc", "tabnet_nosc", "rf_nosc", "svm_nosc", "modig", "mrngcn", "mtgcn", "emogi", "hgdc"]
print("prk")
test(prk, data, 100, pred_files_list, index_list)
print("avepk")
test(avepk, data, 100, pred_files_list, index_list)
print("aupr")
test(average_precision_score, data, None, pred_files_list, index_list)

50 50 50 50 50 50 50 50 50 50
prk


,OncoKB,ONGene,NCG7,Bailey,CancerMine,CCGD,TSGene,IntOGen,MTGCN_OKB,MTGCN_NCG
xgb_nosc,0.2072,0.1138,0.5514,0.1218,0.8074,0.2926,0.2186,0.0926,0.2048,0.0000
mlp_nosc,0.1296,0.0920,0.6636,0.1214,0.7230,0.3488,0.1452,0.1012,0.1294,0.0000
tabnet_nosc,0.1098,0.1016,0.4466,0.0598,0.7232,0.2060,0.1504,0.0620,0.1062,0.0002
rf_nosc,0.2156,0.1586,0.4156,0.0794,0.8768,0.3166,0.2870,0.1020,0.2062,0.0000
svm_nosc,0.1818,0.0868,0.5374,0.0900,0.7114,0.2048,0.1320,0.0666,0.1818,0.0000
modig,0.0992,0.0790,0.5254,0.0792,0.7434,0.2050,0.1326,0.0746,0.0988,0.0000
mrngcn,0.0196,0.0242,0.1684,0.0000,0.4208,0.1022,0.0346,0.0000,0.0104,0.0000
mtgcn,0.1420,0.1320,0.6294,0.1290,0.7886,0.3476,0.1574,0.1142,0.1420,0.0000
emogi,0.1808,0.2130,0.3760,0.0496,0.8694,0.3012,0.2532,0.1156,0.1808,0.0000
hgdc,0.1736,0.1356,0.6872,0.1712,0.8024,0.3624,0.1510,0.1304,0.1736,0.0000


,OKB+NCG,OKB+IntOGen,ONG+TSG,OKB+NCG+ONG,OKB+NCG+ONG+TSG,OKB+Int+NCG,OKB+NCG+ONG+TSG+Int,OKB+NCG+ONG+TSG+CM+CCGD,OKB+NCG+ONG+TSG+CM+CCGD+Int
xgb_nosc,0.6160,0.2574,0.2960,0.6650,0.7588,0.6340,0.7766,0.94,0.94
mlp_nosc,0.6928,0.1870,0.2242,0.7222,0.7846,0.7096,0.8012,0.93,0.93
tabnet_nosc,0.4836,0.1470,0.2296,0.5290,0.6022,0.4896,0.6080,0.94,0.94
rf_nosc,0.4892,0.2576,0.3942,0.5380,0.6804,0.5092,0.7004,0.94,0.94
svm_nosc,0.6084,0.2096,0.2104,0.6418,0.6946,0.6122,0.6984,0.94,0.94
modig,0.5516,0.1416,0.1990,0.5898,0.6438,0.5580,0.6502,0.55,0.55
mrngcn,0.1776,0.0196,0.0582,0.2010,0.2342,0.1776,0.2342,0.61,0.61
mtgcn,0.6574,0.2066,0.2520,0.6968,0.7578,0.6758,0.7736,0.95,0.95
emogi,0.4534,0.2428,0.3986,0.5666,0.6854,0.4734,0.6954,0.93,0.93
hgdc,0.7160,0.2462,0.2468,0.7550,0.7958,0.7296,0.8090,0.97,0.97


avepk


,OncoKB,ONGene,NCG7,Bailey,CancerMine,CCGD,TSGene,IntOGen,MTGCN_OKB,MTGCN_NCG
xgb_nosc,0.044520,0.012219,0.346117,0.029033,0.695259,0.091808,0.051911,0.013884,0.043912,0.000000
mlp_nosc,0.013106,0.007915,0.463392,0.027810,0.522383,0.106757,0.018097,0.011611,0.013078,0.000000
tabnet_nosc,0.015027,0.014148,0.220224,0.010397,0.546306,0.052846,0.027047,0.006507,0.014339,0.000204
rf_nosc,0.054743,0.033567,0.184526,0.006116,0.811967,0.118076,0.115618,0.023251,0.052015,0.000000
svm_nosc,0.034561,0.008120,0.326303,0.015590,0.509363,0.036465,0.016976,0.006598,0.034561,0.000000
modig,0.011682,0.007975,0.326774,0.013668,0.556034,0.047141,0.018439,0.006482,0.011615,0.000000
mrngcn,0.001089,0.000916,0.032362,0.000000,0.213602,0.010546,0.003443,0.000000,0.000258,0.000000
mtgcn,0.016696,0.014467,0.408009,0.021941,0.616215,0.108851,0.020389,0.013047,0.016696,0.000000
emogi,0.053234,0.062689,0.169629,0.003101,0.798142,0.122908,0.064792,0.016360,0.053234,0.000000
hgdc,0.022397,0.014647,0.462844,0.034834,0.638463,0.106893,0.019342,0.016638,0.022397,0.000000


,OKB+NCG,OKB+IntOGen,ONG+TSG,OKB+NCG+ONG,OKB+NCG+ONG+TSG,OKB+Int+NCG,OKB+NCG+ONG+TSG+Int,OKB+NCG+ONG+TSG+CM+CCGD,OKB+NCG+ONG+TSG+CM+CCGD+Int
xgb_nosc,0.425878,0.076108,0.088522,0.483182,0.612542,0.452925,0.645144,0.886461,0.886461
mlp_nosc,0.488620,0.028869,0.041002,0.520733,0.597585,0.511906,0.623244,0.872301,0.872301
tabnet_nosc,0.251751,0.025620,0.058705,0.295729,0.375429,0.258430,0.383517,0.889618,0.889618
rf_nosc,0.265886,0.092517,0.199078,0.328177,0.537836,0.306258,0.595603,0.915617,0.915617
svm_nosc,0.398388,0.047978,0.040092,0.432808,0.491875,0.405301,0.499733,0.871868,0.871868
modig,0.353202,0.020955,0.040346,0.392187,0.448359,0.360631,0.456815,0.356818,0.356818
mrngcn,0.036351,0.001089,0.005775,0.044422,0.061692,0.036351,0.061692,0.407972,0.407972
mtgcn,0.436283,0.035793,0.050561,0.476701,0.544927,0.458794,0.567870,0.897983,0.897983
emogi,0.232226,0.084562,0.166480,0.351262,0.489469,0.251017,0.504312,0.895673,0.895673
hgdc,0.491530,0.048213,0.046441,0.530855,0.577365,0.517291,0.604396,0.924448,0.924448


aupr


ValueError: Input contains NaN.

## Newly predicted cancer driver genes

In [22]:
xgb_pred_files = sorted(glob(f"{BASE_DIR}/crossvalid/xgb/MF+METH+GE+SYS+TOPO+scRaw_all+nonzero_mean/preds/pred_*.txt"))[:50]

pred = []
for pf in xgb_pred_files:
    pred.append(np.loadtxt(pf, delimiter="\t"))

pred = np.stack(pred, axis=0).mean(0)
df = pd.DataFrame(pred, index=genes, columns=["Pred"])
# df["Pred"] = pred
df["NCG"] = data.y_ncg7.bool().numpy()
df["OncoKB"] = data.y_oncokb_latest.bool().numpy()
df["ONGene"] = data.y_ongene.bool().numpy()
df["TSGene"] = data.y_tsgene.bool().numpy()
df["CancerMine"] = data.y_cancermine.bool().numpy()
df["CCGD_A"] = data.y_ccgda.bool().numpy()
df["Overlapped"] = (data.y_oncokb_latest.bool() | data.y_ncg7.bool() | data.y_ongene.bool() | data.y_tsgene.bool() | data.y_cancermine.bool() | data.y_ccgda.bool()).numpy()
df["label"] = (data.train_mask | data.val_mask | data.test_mask).numpy()
df_100 = df[~df["label"]].sort_values(by="Pred", ascending=False).reset_index().iloc[:100]
df_100[["index", "Overlapped", "NCG", "OncoKB", "ONGene", "TSGene", "CancerMine", "CCGD_A"]].to_csv("../supplementary/file3_newly_predicted_genes_top100.csv", index=False)
df_100[~df_100["Overlapped"]]

,index,Granulocyte-monocyte progenitor,Hematopoietic stem cell,Red blood cell,Naive CD4 T cell,Treg cell,CD16 monocyte,Common lymphoid progenitor,Cycling macrophage,MHCII high CD14 monocyte,...,"(SYS, ppin_betweenness)","(SYS, ppin_clustering)","(SYS, complexes)","(SYS, mirna)","(SYS, ohnolog)","(SYS, essentiality_oneCellLine)","(SYS, ppin_hub)",Pred,Independent,label
3,LRP2,0.032944,-0.010213,-0.024326,-0.116756,-0.037157,0.088186,0.002270,-0.094327,0.026282,...,0.242124,0.014113,1.342720,-0.059791,0.010957,0.000435,0.061357,0.999089,False,False
30,PRG2,-0.008448,0.127119,0.034102,0.138319,-0.037909,0.076776,-0.044539,0.060454,-0.086786,...,0.195845,0.011351,3.471067,-0.157301,-0.011052,0.000499,0.048365,0.997924,False,False
51,CAND1,-0.014866,-0.002338,-0.026983,-0.145721,0.010220,-0.046848,0.026054,-0.044845,0.026922,...,0.398069,0.071663,3.415182,0.111882,0.005008,-0.000149,0.107976,0.996617,False,False
72,PLXNA4,0.028955,0.105560,-0.000716,0.233267,0.151144,0.080333,0.031207,0.013617,-0.006582,...,-0.217772,0.011674,3.731829,0.129636,0.009459,0.000026,-0.051092,0.995548,False,False
73,SRRM2,0.035016,-0.040971,-0.028370,0.485697,0.241394,0.021449,0.034559,0.025545,0.083451,...,0.328361,0.057805,1.706098,0.044588,-0.007140,-0.000064,0.068249,0.995511,False,False
79,TUBA1A,0.009466,-0.060192,0.022330,0.428114,0.319030,0.001885,-0.051837,-0.007835,0.110899,...,0.403920,0.061603,2.362871,-0.047749,0.006194,0.000596,0.096335,0.995072,False,False
90,SH3KBP1,-0.008349,0.001138,-0.022703,0.291791,0.288933,-0.036401,0.061206,0.037204,0.091592,...,0.382400,0.041388,2.297559,-0.171550,0.009769,0.001223,0.096837,0.994546,False,False
